In [1]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb
# %run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Feature_Importance_Plot_Random_Forest_Regressor.ipynb

In [ ]:
# def Ensemble_Bagging_Regression(X_train,X_test,y_train,y_test,filtered_data):
#     # Define the objective function for Bayesian optimization
#     def objective(params):
#         n_estimators, max_depth, min_samples_split, min_samples_leaf = params

#         base_regressor = DecisionTreeRegressor(        
#                     max_depth=max_depth,
#                     min_samples_split=min_samples_split,
#                     min_samples_leaf=min_samples_leaf,
#                     random_state=0)
#         model = BaggingRegressor(estimator=base_regressor, n_estimators=n_estimators, random_state=42)

#         model.fit(X_train, y_train)
#         predict_train = model.predict(X_train)
#         r2 = r2_score(y_train, predict_train)

#         return -r2  # Negate the r2 score for minimization

#     # Define the hyperparameter space
#     param_space = [
#         Integer(5, 50, name="n_estimators"),
#         Integer(1, 50, name="max_depth"),
#         Integer(2, 20, name="min_samples_split"),
#         Integer(1, 20, name="min_samples_leaf")
#     ]

#     # Perform Bayesian optimization
#     results = gp_minimize(objective, param_space, n_calls=50, random_state=0)

#     # Extract the best hyperparameters
#     n_estimators, max_depth, min_samples_split, min_samples_leaf= results.x

#     # Create and train the model with the best hyperparameters
#     best_model = BaggingRegressor(
#         estimator = DecisionTreeRegressor(                    
#                     max_depth=max_depth,
#                     min_samples_split=min_samples_split,
#                     min_samples_leaf=min_samples_leaf,
#                     random_state=0),
#         n_estimators=n_estimators,
#         random_state=42
#     )
#     best_model.fit(X_train, y_train)

#     # Make predictions and create DataFrames for visualization
#     predict_train = best_model.predict(X_train)
#     predict_test = best_model.predict(X_test)
#     train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
#     test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)
#     combined_df = pd.concat([train_df, test_df])
#     combined_df = combined_df.sort_index()
#     print(combined_df)

#     # Calculate and print R2 scores
#     r2_score_train = round(r2_score(y_train, predict_train), 2)
#     r2_score_test = round(r2_score(y_test, predict_test), 2)
#     print('R2 value of train data:', r2_score_train)
#     print('R2 value of test data:', r2_score_test)

#     # Visualization
#     if (v == 'dCn'):
#         unit = 'dCn'
#     else:
#         unit = 'pm_conc'
#     Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit)
#     qq_plot(test_df, v, unit)

#     # Return the model and the best hyperparameters
#     return best_model, results.x,r2_score_test

#     #Feature_Importance_Random_Forest_Regressor(model, filtered_data, v, unit)

In [ ]:
def Ensemble_Bagging_Regression(X_train, X_test, y_train, y_test, filtered_data):
    # Function to be optimized by Bayesian Optimization
    def bagging_regressor_cv(n_estimators, max_samples, max_features):
        # Ensure parameters are integer type
        n_estimators = int(n_estimators)
        max_samples = max_samples
        max_features = max_features
        
        # Define and fit the model
        model = BaggingRegressor(
            estimator=DecisionTreeRegressor(),
            n_estimators=n_estimators,
            max_samples=max_samples,
            max_features=max_features,
            random_state=42
        )
        model.fit(X_train, y_train)

        # Predict and calculate R2 score
        predict_test = model.predict(X_test)
        r2_score_test = metrics.r2_score(y_test, predict_test)
        return r2_score_test

    # Setting ranges for hyperparameters
    pbounds = {
        'n_estimators': (10, 100),
        'max_samples': (0.5, 1.0),
        'max_features': (0.5, 1.0)
    }

    # Bayesian Optimization
    optimizer = BayesianOptimization(
        f=bagging_regressor_cv,
        pbounds=pbounds,
        random_state=42
    )
    optimizer.maximize(init_points=5, n_iter=10)

    # Optimal parameters
    params = optimizer.max['params']
    params['n_estimators'] = int(params['n_estimators'])
    params['max_samples'] = np.round(params['max_samples'],4)
    params['max_features'] = np.round(params['max_features'],4)
    
    print("Optimal Parameters:", params)

    # Create and train model with optimal parameters
    best_model = BaggingRegressor(
        estimator=DecisionTreeRegressor(),
        n_estimators=params['n_estimators'],
        max_samples=params['max_samples'],
        max_features=params['max_features'],
        random_state=42
    )
    best_model.fit(X_train, y_train)

    # Predictions
    predict_train = best_model.predict(X_train)
    predict_test = best_model.predict(X_test)

    # DataFrame Creation
    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    # Concatenating and Sorting
    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()

    # R2 Score
    r2_score_train = round(metrics.r2_score(y_train, predict_train), 2)
    r2_score_test = round(metrics.r2_score(y_test, predict_test), 2)
    print('R2 Score Train:', r2_score_train)
    print('R2 Score Test:', r2_score_test)

    return best_model,params,r2_score_test